# Banking Loan classification 
---

This notebook is used to build the **ML model** locally. The main objective here is to get the model object to host as a SageMaker endpoint in the next section.


The notebook uses the data from [Kaggle dataset](https://www.kaggle.com/datasets/kapturovalexander/credit-classification-for-banks).

The data is under [CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/).

<div class="alert alert-block alert-info">
    This notebook has been tested and run on <b>Python 3 (Data Science 2.0)</b> kernel and on <b>ml.m5.xlarge</b> instance type.
</div>

## Setup
---

Install and update some libraries for the model.

In [2]:
!pip install xgboost imblearn catboost lightgbm xgboost pyxlsb --quiet
!pip install threadpoolctl==3.2.0


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


Logging definition

In [3]:
import logging

logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger.setLevel(logging.INFO)

In [4]:
import pandas as pd
import numpy as np
import os, sys, gc
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
import imblearn
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score

10:48:48 INFO:NumExpr defaulting to 4 threads.


## Get the data
---

Load the dataset and put them into `data` folder.

In [5]:
train_file_nm = 'Training.xlsb'
test_file_nm = 'Test.xlsb'

train_df = pd.read_excel(io=f'data/{train_file_nm}')
test_df = pd.read_excel(io=f'data/{test_file_nm}')

logging.info(f'Training data shape => {train_df.shape}')
logging.info(f'Testing data shape => {test_df.shape}')

10:49:20 INFO:Training data shape => (89734, 26)
10:49:20 INFO:Testing data shape => (38405, 26)


Due to large number of columns, we want to explore all of them.

In [6]:
pd.set_option('display.max_columns', None)
train_df.head(3)

,ID,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,MARKER
0,1,0.198778,0.099389,0.00,799.90,1.777556,0.888778,13,3.49,Woman,33.750000,Mogilev region,132,NE employee,Secondary education (plus special education),Married,2 Two,property,No,Works,Yes,Yes,No,Yes,No,0
1,2,0.043000,0.021264,49.97,173.03,0.384511,0.190143,13,3.49,Woman,33.250000,Minsk region,6,NE employee,Secondary education (plus special education),Married,2 Two,otherwise,No,Works,Yes,Yes,Yes,Yes,No,0
2,3,0.067073,0.067073,0.00,329.90,0.599818,0.599818,13,3.49,Woman,38.583333,Vitebsk region,71,NE employee,Secondary education (plus special education),Divorced/widow,1 One,property,No,No couple,No,Yes,No,Yes,No,0


## Data Checking
---

Check some of the data columns

In [7]:
logging.info(train_df.columns)

10:49:21 INFO:Index(['ID', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'MARKER'],
      dtype='object')


In [8]:
for obj_col in list(train_df.select_dtypes(include=['object']).columns):
    logging.info(f'for column {obj_col} has unique values as => {train_df[obj_col].unique()}')

10:49:21 INFO:for column I has unique values as => ['Woman' 'Man']
10:49:21 INFO:for column K has unique values as => ['Mogilev region' 'Minsk region' 'Vitebsk region' 'Brest region' 'Minsk'
 'Grodno region' 'Gomel region']
10:49:21 INFO:for column M has unique values as => ['NE employee' 'Pensioner' 'Head/Deputy head (organiz.)'
 'Head/Deputy head (division)' 'Enterpreneur']
10:49:21 INFO:for column N has unique values as => ['Secondary education (plus special education)'
 'Higher education (one or more)' 'Incomplete higher education'
 'Primary or lower secondary education']
10:49:21 INFO:for column O has unique values as => ['Married' 'Divorced/widow' 'Single/unmarried' 'Cohabitation']
10:49:21 INFO:for column P has unique values as => ['2 Two' '1 One' '0 Zero' '3 Three' 'More than 3']
10:49:21 INFO:for column Q has unique values as => ['property' 'otherwise' 'rent/hire']
10:49:21 INFO:for column R has unique values as => ['No' 'Yes']
10:49:21 INFO:for column S has unique values as =

Based on the list of values, we can guess some of the column names. Let's rename columns for better understanding.

In [9]:
train_df.rename(columns={'I': 'Sex'}, inplace=True)
train_df.rename(columns={'K': 'Region'}, inplace=True)
train_df.rename(columns={'M': 'Job_title'}, inplace=True)
train_df.rename(columns={'N': 'Education'}, inplace=True)
train_df.rename(columns={'O': 'Marriage'}, inplace=True)
train_df.rename(columns={'P': 'Children'}, inplace=True)
train_df.rename(columns={'Q': 'Property'}, inplace=True)
train_df.rename(columns={'S': 'Employment_status'}, inplace=True)

test_df.rename(columns={'I': 'Sex'}, inplace=True)
test_df.rename(columns={'K': 'Region'}, inplace=True)
test_df.rename(columns={'M': 'Job_title'}, inplace=True)
test_df.rename(columns={'N': 'Education'}, inplace=True)
test_df.rename(columns={'O': 'Marriage'}, inplace=True)
test_df.rename(columns={'P': 'Children'}, inplace=True)
test_df.rename(columns={'Q': 'Property'}, inplace=True)
test_df.rename(columns={'S': 'Employment_status'}, inplace=True)

In [10]:
logging.info(train_df.columns)

10:49:21 INFO:Index(['ID', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Sex', 'J', 'Region', 'L',
       'Job_title', 'Education', 'Marriage', 'Children', 'Property', 'R',
       'Employment_status', 'T', 'U', 'V', 'W', 'X', 'MARKER'],
      dtype='object')


In [11]:
logging.info(f'\n{train_df.MARKER.value_counts()}')
logging.info(f'\n{train_df.MARKER.value_counts() / train_df.shape[0]}')

10:49:21 INFO:
0    89392
1      342
Name: MARKER, dtype: int64
10:49:21 INFO:
0    0.996189
1    0.003811
Name: MARKER, dtype: float64


Are there any missing values in the dataset?

In [12]:
logging.info(f'\n{train_df.isna().sum()}')

10:49:21 INFO:
ID                   0
A                    0
B                    0
C                    0
D                    0
E                    0
F                    0
G                    0
H                    0
Sex                  0
J                    0
Region               0
L                    0
Job_title            0
Education            0
Marriage             0
Children             0
Property             0
R                    0
Employment_status    0
T                    0
U                    0
V                    0
W                    0
X                    0
MARKER               0
dtype: int64


In [13]:
logging.info(train_df.info())

10:49:21 INFO:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89734 entries, 0 to 89733
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 89734 non-null  int64  
 1   A                  89734 non-null  float64
 2   B                  89734 non-null  float64
 3   C                  89734 non-null  float64
 4   D                  89734 non-null  float64
 5   E                  89734 non-null  float64
 6   F                  89734 non-null  float64
 7   G                  89734 non-null  int64  
 8   H                  89734 non-null  float64
 9   Sex                89734 non-null  object 
 10  J                  89734 non-null  float64
 11  Region             89734 non-null  object 
 12  L                  89734 non-null  int64  
 13  Job_title          89734 non-null  object 
 14  Education          89734 non-null  object 
 15  Marriage           89734 non-null  object 
 16  Children           897

## Prepare data for model building
---

In this section, I will split the data and create several models.

In [14]:
cols_to_drop = ['ID', 'MARKER']
target_col = 'MARKER'

# gather X and y from train dataset
X_train = train_df.drop(cols_to_drop, axis=1, inplace=False)
y_train = train_df[target_col]

# gather ID and y from test dataset
y_test = test_df[target_col]
id_test = test_df['ID']
test_df.drop(cols_to_drop, axis=1, inplace=True)

In [15]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(test_df, drop_first=True)

In [16]:
assert (X_train.columns == X_test.columns).all()
logging.info('all matches!')

10:49:21 INFO:all matches!


Check the correlation matrix

In [17]:
corr_df = X_train.corr()
corr_df.style.background_gradient(cmap='coolwarm')

,A,B,C,D,E,F,G,H,J,L,Sex_Woman,Region_Gomel region,Region_Grodno region,Region_Minsk,Region_Minsk region,Region_Mogilev region,Region_Vitebsk region,Job_title_Head/Deputy head (division),Job_title_Head/Deputy head (organiz.),Job_title_NE employee,Job_title_Pensioner,Education_Incomplete higher education,Education_Primary or lower secondary education,Education_Secondary education (plus special education),Marriage_Divorced/widow,Marriage_Married,Marriage_Single/unmarried,Children_1 One,Children_2 Two,Children_3 Three,Children_More than 3,Property_property,Property_rent/hire,R_Yes,Employment_status_Pensioner,Employment_status_Student,Employment_status_Unemployed,Employment_status_Works,T_Yes,U_Yes,V_No,V_Yes,W_Yes,X_Yes
A,1.000000,0.787760,0.279254,0.390606,0.648925,0.473056,-0.324324,-0.229381,0.154316,0.011154,0.149300,0.016255,0.015170,-0.037219,-0.011192,0.024513,-0.004739,-0.051073,-0.016919,-0.051990,0.179834,-0.006163,0.005013,0.069637,0.002221,0.006876,-0.019012,-0.069120,-0.071544,-0.027904,-0.012326,0.040525,-0.028780,-0.034517,0.090146,-0.005758,-0.093160,0.028238,0.037167,-0.009326,0.084198,-0.076897,-0.080715,0.092369
B,0.787760,1.000000,0.282321,0.371655,0.471430,0.657744,-0.317508,-0.212887,0.081070,-0.009512,0.053479,0.022444,0.009723,-0.023171,-0.017510,0.015359,-0.006042,-0.043233,-0.013232,-0.030038,0.123836,0.005617,0.004599,0.059332,0.200160,-0.280887,0.183912,-0.081647,-0.097244,-0.026426,-0.007639,0.006141,-0.013065,-0.053565,0.025072,0.005375,0.058494,-0.306655,0.031703,-0.004206,0.078257,-0.071820,-0.085488,0.081038
C,0.279254,0.282321,1.000000,0.495777,0.318026,0.322362,0.019881,-0.211125,-0.002666,0.011004,-0.048296,0.009919,-0.034060,0.058811,0.003578,0.023828,-0.023780,0.035767,0.048484,-0.053151,-0.020952,0.000166,-0.012127,-0.085327,-0.025228,0.009752,0.009789,-0.017796,-0.010412,-0.008427,-0.004082,0.016871,-0.009459,0.032653,-0.002500,-0.004144,0.006764,0.005782,-0.013914,-0.004304,0.034681,-0.031627,-0.045180,-0.082019
D,0.390606,0.371655,0.495777,1.000000,0.733874,0.702949,0.336855,-0.041913,0.030375,0.032271,-0.037662,0.004560,-0.046608,0.087795,0.014765,0.024001,-0.035309,0.042866,0.059462,-0.055380,-0.030965,-0.000877,-0.007476,-0.070801,-0.029684,0.037277,-0.028963,-0.018335,-0.012231,0.003689,-0.004126,0.035775,-0.021384,0.046840,-0.002467,-0.004682,0.001162,0.034613,-0.007673,-0.000221,0.017212,-0.017723,-0.023294,0.225930
E,0.648925,0.471430,0.318026,0.733874,1.000000,0.782391,0.279598,0.021942,0.113036,-0.006566,0.142138,0.012407,0.011609,-0.032243,-0.021496,0.040214,-0.001851,-0.047668,-0.007088,-0.040594,0.150497,-0.003167,0.008041,0.066227,-0.005840,0.015785,-0.026452,-0.055630,-0.058841,-0.017364,-0.014672,0.037401,-0.024176,-0.029782,0.059443,-0.006253,-0.084032,0.043976,0.024644,-0.006263,0.058683,-0.054196,-0.058909,0.187421
F,0.473056,0.657744,0.322362,0.702949,0.782391,1.000000,0.275341,0.044503,0.042328,-0.027840,0.042214,0.018135,0.006850,-0.019260,-0.027642,0.029893,-0.001798,-0.040214,-0.003594,-0.021675,0.099746,0.008931,0.007186,0.056981,0.192573,-0.272640,0.177479,-0.068677,-0.085851,-0.016672,-0.009440,0.005554,-0.009132,-0.049549,-0.000412,0.004925,0.075702,-0.299199,0.020192,-0.001466,0.051572,-0.047948,-0.064461,0.171696
G,-0.324324,-0.317508,0.019881,0.336855,0.279598,0.275341,1.000000,0.597774,-0.058046,-0.036009,-0.012810,-0.006097,-0.010101,0.004886,-0.013200,0.008231,0.019025,0.002652,0.006956,0.017980,-0.032574,0.008015,0.007184,0.019647,-0.002649,0.002047,-0.007451,0.025944,0.014158,0.016207,0.003105,-0.004767,0.008236,0.004718,-0.035248,-0.000060,0.010776,0.010397,-0.010697,-0.002203,-0.052516,0.046035,0.040366,0.106804
H,-0.229381,-0.212887,-0.211125,-0.041913,0.021942,0.044503,0.597774,1.000000,-0.062062,-0.046920,-0.003752,-0.001776,-0.008228,-0.044233,-0.017166,-0.022835,0.067950,-0.029315,-0.025651,0.058586,-0.020049,0.010949,0.018608,0.108377,0.027386,-0.038627,0.016588,0.031663,0.010871,0.017250,0.008316,-0.023999,0.023077,-0.013594,-0.028798,0.001449,-

In [18]:
upper = corr_df.where(np.triu(np.ones(corr_df.shape), k=1).astype(bool))
# just for simplicity, I will remove any columns with correlation > 0.7
to_drop = [col for col in upper.columns if any(upper[col] > 0.7)]
logger.info(to_drop)

10:49:22 INFO:['B', 'E', 'F']


In [19]:
X_train.drop(to_drop, axis=1, inplace=True)
X_test.drop(to_drop, axis=1, inplace=True)

In [20]:
logger.info(f'Train dataset => {X_train.shape}, {y_train.shape}')
logger.info(f'Test dataset => {X_test.shape}, {y_test.shape}')

10:49:22 INFO:Train dataset => (89734, 41), (89734,)
10:49:22 INFO:Test dataset => (38405, 41), (38405,)


## SMOTE
---

Given the exploration above, we can see the imbalance of class (this is normal for loan / fraud classification in banking industry. We have many ways to deal with these but, again, we opt to use `SMOTE` here.

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [22]:
logger.info(f'Train dataset => {X_train.shape}, {y_train.shape}')
logger.info(f'Validation dataset => {X_val.shape}, {y_val.shape}')

10:49:22 INFO:Train dataset => (71787, 41), (71787,)
10:49:22 INFO:Validation dataset => (17947, 41), (17947,)


In [23]:
logger.info(f'{imblearn.__version__}')

10:49:22 INFO:0.11.0


In [24]:
sm = imblearn.over_sampling.BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train, y_train)

In [25]:
logger.info(f'Train dataset => {X_train_resampled.shape}, {y_train_resampled.shape}')
logger.info(f'Validation dataset => {X_val.shape}, {y_val.shape}')

10:49:23 INFO:Train dataset => (143008, 41), (143008,)
10:49:23 INFO:Validation dataset => (17947, 41), (17947,)


## Build XGBoost model locally


In [26]:
xgb = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)
xgb.fit(X_train_resampled, y_train_resampled, eval_set=[(X_val, y_val)])

[0]	validation_0-logloss:0.60681
[1]	validation_0-logloss:0.53882
[2]	validation_0-logloss:0.47932
[3]	validation_0-logloss:0.42942
[4]	validation_0-logloss:0.38854
[5]	validation_0-logloss:0.34958
[6]	validation_0-logloss:0.31620
[7]	validation_0-logloss:0.28744
[8]	validation_0-logloss:0.26246
[9]	validation_0-logloss:0.24027
[10]	validation_0-logloss:0.22068
[11]	validation_0-logloss:0.20348
[12]	validation_0-logloss:0.18731
[13]	validation_0-logloss:0.17292
[14]	validation_0-logloss:0.15971
[15]	validation_0-logloss:0.14770
[16]	validation_0-logloss:0.13715
[17]	validation_0-logloss:0.12739
[18]	validation_0-logloss:0.11888
[19]	validation_0-logloss:0.11126
[20]	validation_0-logloss:0.10470
[21]	validation_0-logloss:0.09840
[22]	validation_0-logloss:0.09204
[23]	validation_0-logloss:0.08638
[24]	validation_0-logloss:0.08173
[25]	validation_0-logloss:0.07634
[26]	validation_0-logloss:0.07267
[27]	validation_0-logloss:0.06838
[28]	validation_0-logloss:0.06469
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [27]:
y_pred_val = xgb.predict_proba(X_val)
y_pred_test = xgb.predict_proba(X_test)
roc_auc_val = roc_auc_score(y_val, y_pred_val[:, 1])
roc_auc_test = roc_auc_score(y_test, y_pred_test[:, 1])
logger.info(f'Validation ROC AUC Score => {roc_auc_val}')
logger.info(f'Test ROC AUC Score => {roc_auc_test}')

10:49:38 INFO:Validation ROC AUC Score => 0.848234589612201
10:49:38 INFO:Test ROC AUC Score => 0.8531040417743949


In [28]:
y_pred_val_label = xgb.predict(X_val)
y_pred_test_label = xgb.predict(X_test)
print(classification_report(y_val, y_pred_val_label))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17888
           1       0.11      0.08      0.10        59

    accuracy                           0.99     17947
   macro avg       0.55      0.54      0.55     17947
weighted avg       0.99      0.99      0.99     17947



In [29]:
print(
    classification_report(y_test, y_pred_test_label)
)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     38259
           1       0.11      0.07      0.08       146

    accuracy                           0.99     38405
   macro avg       0.55      0.53      0.54     38405
weighted avg       0.99      0.99      0.99     38405



## Save XGBoost model
---

We will save the `XGBoost` model locally as well as upload to **Amazon S3** bucket.

In [30]:
import pickle
file_name = "banking_loan_classifier.pkl"
pickle.dump(xgb, open(file_name, "wb"))